In [41]:
# Never forget to firs check the .../robot.txt of every website you wish to scrap/crawl over

In [2]:
# load the libraries you'll need
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import scrapy

In [3]:
# Save your html file with this function that way you don't male repeated requests to a website
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

# Open it back up with this function
def open_html(path):
    with open(path, 'rb') as f:
        return f.read()

In [4]:
# This is the prefix of the website for later use
prefix = "http://www.ifar.org/"

In [5]:
url = 'http://www.ifar.org/catalogues_raisonnes.php?alpha=&searchtype=artist&published=1&inPrep=1&artist=&author='
path = 'C:/Users/ramon/OneDrive/INSEAD/auction_proj'

In [10]:
# create a conection with the website
r = requests.get(url)
print(r.content[:100])

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtm'


In [11]:
# assign the content of the website to a new variable using the method .content then save to not make more requests
save_html(r.content, 'auctions_html')

In [6]:
# Load back in the hmtl of the page
html = open_html('auctions_html')

In [7]:
# instantiate a BeatifulSoup object with the content and the type of parser you'll use
soup = BeautifulSoup(html, "html.parser")

In [8]:
# Inspect soup to make sure data was loaded successfully
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html>
<head>
<title>International Foundation for Art Research (IFAR)-Catalogues Raisonnés</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="International Foundation for Art Research - Catalogues raisonnés — scholarly compilations of an artist's body of work — are critical tools for researching the provenance and attribution of artwork. Here are two new electronic resources — a database of published catalogues raisonnés and one of catalogues in preparation. They can be searched separately or together and are updated regularly." name="description"/>
<meta content="Art, Stolen Art, Art Research, Provenance, Art Law, Catalogues Raisonnes" name="keywords"/>
<meta content="no-cache" http-equiv="cache-control"/>
<meta content="no-cache" http-equiv="pragma"/>
<meta content="noarchive" name="googlebot"/>
<meta content="0" http-equi

In [9]:
# Select the rows of the tables that contain the data
rows = soup.select('tr')
rows = rows[1:]

In [10]:
len(rows)

2918

In [15]:
#Inspect a row
row = rows[0]

In [16]:
name = row.select_one('.g_center_name').get_text('|').strip()

In [17]:
print(name)

Aachen, Hans von|click to learn more


In [18]:
allsides_page = row.select_one('.g_center_name a')['href']
allsides_page = prefix + allsides_page

print(allsides_page)

http://www.ifar.org/artist_name.php?nameid=1563&published=1&inPrep=1&author=


In [19]:
birth_year = int(row.select_one('.g_center_birth_year').get_text('|').strip())
#birth_year = int(birth_year)
print(birth_year)

1552


In [20]:
birth_place = row.select_one('.g_center_birth_place').get_text('|').strip()
birth_place

'Germany'

In [21]:
death_year = row.select_one('.g_center_death_year').get_text('|').strip()
death_year = int(death_year)
print(death_year)

1615


In [22]:
death_year = int(row.select_one('.g_center_death_year').text)
#death_year = int(death_year)
print(death_year)

1615


In [23]:
death_place = row.select_one('.g_center_death_place').get_text('|').strip()
death_place

'Czech Republic'

In [11]:
data= []

for ro in rows:
    d = dict()
    
    d['name'] = ro.select_one('.g_center_name').get_text('|').strip()
    
    d['allsides_page'] = 'http://www.ifar.org/' + ro.select_one('.g_center_name a')['href']
    
    d['birth_year'] = ro.select_one('.g_center_birth_year').text
    
    d['birth_place'] = ro.select_one('.g_center_birth_place').get_text('|').strip()

    d['death_year'] = ro.select_one('.g_center_death_year').text
    
    d['death_place'] = ro.select_one('.g_center_death_place').get_text('|').strip()
    
    data.append(d)

In [12]:
print(data[3])

{'name': 'Abdessemed, Adel|click to learn more', 'allsides_page': 'http://www.ifar.org/artist_name.php?nameid=3073&published=1&inPrep=1&author=', 'birth_year': '1971', 'birth_place': 'Algeria', 'death_year': '', 'death_place': ''}


In [30]:
from copy import deepcopy
from tqdm import tqdm_notebook
from time import sleep

In [13]:
data

[{'name': 'Aachen, Hans von|click to learn more',
  'allsides_page': 'http://www.ifar.org/artist_name.php?nameid=1563&published=1&inPrep=1&author=',
  'birth_year': '1552',
  'birth_place': 'Germany',
  'death_year': '1615',
  'death_place': 'Czech Republic'},
 {'name': 'Aalto, Alvar|click to learn more',
  'allsides_page': 'http://www.ifar.org/artist_name.php?nameid=1396&published=1&inPrep=1&author=',
  'birth_year': '1898',
  'birth_place': 'Finland',
  'death_year': '1976',
  'death_place': 'Finland'},
 {'name': 'Abbati, Giuseppe|click to learn more',
  'allsides_page': 'http://www.ifar.org/artist_name.php?nameid=1397&published=1&inPrep=1&author=',
  'birth_year': '1836',
  'birth_place': 'Italy',
  'death_year': '1868',
  'death_place': 'Italy'},
 {'name': 'Abdessemed, Adel|click to learn more',
  'allsides_page': 'http://www.ifar.org/artist_name.php?nameid=3073&published=1&inPrep=1&author=',
  'birth_year': '1971',
  'birth_place': 'Algeria',
  'death_year': '',
  'death_place': '

In [25]:
sel.xpath('//tr/td/a/@href').extract()

['artist_name.php?nameid=1563&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1396&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1397&published=1&inPrep=1&author=',
 'artist_name.php?nameid=3073&published=1&inPrep=1&author=',
 'artist_name.php?nameid=2030&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1991&published=1&inPrep=1&author=',
 'artist_name.php?nameid=2292&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1637&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1849&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1636&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1457&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1210&published=1&inPrep=1&author=',
 'artist_name.php?nameid=2709&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1638&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1866&published=1&inPrep=1&author=',
 'artist_name.php?nameid=1640&published=1&inPrep=1&author=',
 'artist_name.php?nameid

In [32]:
for d in data:
    r = requests.get(d['allsides_page'])
    soup = BeautifulSoup(r.content, 'html.parser')
    
    website = soup.select_one('a')['href']
    d['website'] = website

In [39]:
print(data[90])

{'name': 'Arias Yebra, Amable|click to learn more', 'allsides_page': 'http://www.ifar.org/artist_name.php?nameid=1880&published=1&inPrep=1&author=', 'birth_year': '1927', 'birth_place': 'Spain', 'death_year': '1984', 'death_place': 'Spain'}


In [40]:
webbb = soup.select_one('.www')['href']
print(webbb)

TypeError: 'NoneType' object is not subscriptable

In [51]:
# Doing the column with paths using scrapy

In [22]:
sel = scrapy.Selector(text = html)

In [23]:
ps = sel.xpath('//tr').xpath('./td')
second = ps
second.extract()

['<td class="top_center_name">Name </td>',
 '<td class="top_center_birth_year">Birth Year </td>',
 '<td class="top_center_birth_place">Birth Place </td>',
 '<td class="top_center_death_year">Death Year </td>',
 '<td class="top_center_death_place">Death Place </td>',
 '<td class="g_center_name"><a href="artist_name.php?nameid=1563&amp;published=1&amp;inPrep=1&amp;author=">Aachen, Hans von<span>click to learn more</span></a></td>',
 '<td class="g_center_birth_year">1552</td>',
 '<td class="g_center_birth_place">Germany</td>',
 '<td class="g_center_death_year">1615</td>',
 '<td class="g_center_death_place">Czech Republic</td>',
 '<td class="g_center_name"><a href="artist_name.php?nameid=1396&amp;published=1&amp;inPrep=1&amp;author=">Aalto, Alvar<span>click to learn more</span></a></td>',
 '<td class="g_center_birth_year">1898</td>',
 '<td class="g_center_birth_place">Finland</td>',
 '<td class="g_center_death_year">1976</td>',
 '<td class="g_center_death_place">Finland</td>',
 '<td class=

In [ ]:
for d in data:
    r = requests.get(d['allsides_page'])
    soup = BeautifulSoup(r.content, 'html.parser')
    
    try:
        website = soup.select_one('.www')['href']
        d['website'] = website

In [26]:
data

[{'name': 'Aachen, Hans von|click to learn more',
  'allsides_page': 'http://www.ifar.org/artist_name.php?nameid=1563&published=1&inPrep=1&author=',
  'birth_year': '1552',
  'birth_place': 'Germany',
  'death_year': '1615',
  'death_place': 'Czech Republic'},
 {'name': 'Aalto, Alvar|click to learn more',
  'allsides_page': 'http://www.ifar.org/artist_name.php?nameid=1396&published=1&inPrep=1&author=',
  'birth_year': '1898',
  'birth_place': 'Finland',
  'death_year': '1976',
  'death_place': 'Finland'},
 {'name': 'Abbati, Giuseppe|click to learn more',
  'allsides_page': 'http://www.ifar.org/artist_name.php?nameid=1397&published=1&inPrep=1&author=',
  'birth_year': '1836',
  'birth_place': 'Italy',
  'death_year': '1868',
  'death_place': 'Italy'},
 {'name': 'Abdessemed, Adel|click to learn more',
  'allsides_page': 'http://www.ifar.org/artist_name.php?nameid=3073&published=1&inPrep=1&author=',
  'birth_year': '1971',
  'birth_place': 'Algeria',
  'death_year': '',
  'death_place': '